In [330]:
using Printf
using Plots
using LinearAlgebra # do operacji macierzowych i testowania
using RandomNumbers.MersenneTwisters
using JLD

# NOTES:
# identity: Matrix(I, rows, cols)
# random: rand(rows, cols)
# transpose: matrix'
# one row tmp[1:1, :]
# cats: [A B]=hcat [A; B]=vcat
# vector/matrix length/norm = norm([1 2 3]) !!! norma Frobeniusa
# subarray M[2:3, 4:5]
# eltype(M) type of Matrix elements
# NamedTuple{(:a, :b), Tuple{Matrix, Matrix}}((Matrix{Float64}(I, 5, 5), rand(4, 4))).b

Znajdujemy rozkład macieży a na macieże `Q`(ortogonalną) i `R`(górnotrójkątną). Dzieki temu `rank(A)=rank(QR)=rank(R)`, a `rank(R)` jest prostym policzeniem niezerowych wierszy. Korzystam z metody Householdera, ze względu na lepszą numeryczną poprawność niż metoda Gramma-Schmidta.

In [260]:
function getPrecisionU(T=Float64) return 1/2.0*2.0^-precision(T) end
# https://pl.wikipedia.org/wiki/Rozk%C5%82ad_QR
# oraz https://rosettacode.org/wiki/QR_decomposition
# liczy podmacierz Householdera dla wektora `aₙₓ₁` jako Hₙₓₙ=I-2*(v*vᵀ)/(vᵀ*v) gdzie v = a - |a| * [1₁ 0₂ ... 0ₙ]ᵀ, v!=0
# macierz zwracana spełnia warunek H*a=|a|*e, więc dla a=d*e H=I, gdzie d ∈ R, e = [1₁ 0₂ ... 0ₙ]ᵀ
function _HouseholderMatrix(a) # O(n²)
    local T, n = eltype(a), length(a)
    if (all(x->x==0, a[2:n])) return Matrix{T}(I, n, n) end # zwraca identyczność gdy a=d*e, bo inaczej v=0
    local v = a+(a[1]>=0 ? 1 : -1)*norm(a)*[1 zeros(T, n-1)']' # a±|a|... zamiast a-|a|... zapobiega utracie cyfr znaczących
    #if(v[1] != 0) v = v/v[1] end # normalizacja ze względu na pierwszy element. Dla a⟂e(v[1]=0) nie normalizujemy.
    return Matrix{T}(I, n, n) - (2/(v'*v)[1]) * (v*v') # w mianowniku iloczyn skalarny <v,v> != 0. O(n²)
    end 

# liczy macierz R rozkładu QR (Mₙₓₘ=QₙₓₘRₘₓₘ gdzie Q ortogonana i R górnotrójkątna) metodą odbić Householdera.
# funkcja pivotująca wybierająca wektor o największej normie
function _rankRHouseholder(M)
    # R=...H₂H₁M   Q=H₁H₂... (ale Q jest zbędne)
    local T, n, m = eltype(M), length(M[:, 1]), length(M[1, :]) # rows and columns
    if (n < m) n, m, M = m, n, M' end # transponuj. Rząd kolumnowy i wierszowy są takie same a rk(M)=rk(Mᵀ)=rk(Rᵀ)
    local R = convert(Array{T}, Matrix(M))
    for i in 1:min(m, n-1)
#         display(i);
#         display(R)
#         display(R[i:n, i:i]')
        R[:, i:m] = sortslices(R[:, i:m], dims=2, lt=(x,y)->norm(x[i:n])<norm(y[i:n]), rev=true) # pivot
        R = [Matrix{T}(I, i-1, i-1) zeros(T, i-1, n-i+1); zeros(T, n-i+1, i-1) _HouseholderMatrix(R[i:n, i:i])]*R
#        R[i+1:n, i:i] = zeros(T, n-i)' # eksperyment - co jeśli sztucznie wstawiamy zera
    end
    return R # TODO: liczenie rzędu macierzy górnotrójkątnej
end

# zamienia liczby zmiennoprzecinkowe w macierzy na zera jeśli |x|≦|maxError|
function filterZeros(matrix, maxError)
    return map(x->abs(x)<=abs(maxError) ? 0.0 : x, matrix)
end

# returns number of nonzero rows, when zero row is equal to zeros(row_length)
function countNonzeroRows(matrix)
    local nonzero = 0
    for row in 1:length(matrix[:, 1])
        nonzero += matrix[row, :] == zeros(length(matrix[1, :])) ? 0 : 1
    end
    return nonzero
end

# zwraca rząd macierzy z użyciem 
function rankRevealHouseholder(matrix; maxError = getPrecisionU()*50)
    return countNonzeroRows(filterZeros(_rankRHouseholder(matrix), maxError))
end

function rankReveal

LoadError: syntax: incomplete: premature end of input

In [261]:
#A = [12 -51 4; 6 167 -68; -4 24 -41]
tmp = rand(-10:10, 6, 6); display("tmp:"); display(tmp)
tmp2 = copy(tmp); tmp2[2, :] = -tmp2[5, :]; display("tmp2:"); display(tmp2) # mniejszy rząd wierszowy
tmp3 = copy(tmp2); tmp3[:, 2] = -tmp3[:, 4]; display("tmp3:"); display(tmp3) # mniejszy rząd wierszowy i kolumnowy
#_rankRHouseholder(tmp3)

"tmp:"

6×6 Array{Int64,2}:
 0   4    6  -4  -3    0
 9  -6   -6  -3  -5   -6
 6   6    9   7   8  -10
 3   9  -10   6   2    2
 5   0   -7  -5   2    6
 2  -5   -7  -3  -4    5

"tmp2:"

6×6 Array{Int64,2}:
  0   4    6  -4  -3    0
 -5   0    7   5  -2   -6
  6   6    9   7   8  -10
  3   9  -10   6   2    2
  5   0   -7  -5   2    6
  2  -5   -7  -3  -4    5

"tmp3:"

6×6 Array{Int64,2}:
  0   4    6  -4  -3    0
 -5  -5    7   5  -2   -6
  6  -7    9   7   8  -10
  3  -6  -10   6   2    2
  5   5   -7  -5   2    6
  2   3   -7  -3  -4    5

Problem jest znalezienie rzędu macierzy R, ponieważ nie musi ona być w postaci zredukowanej. Do tego jest algorytm "Rank Revealing QR decomposition" (ang. RRQR), gdzie znajdujemy dodatkową macierz P permutacji wejsciowej macierzy A. Dzięki temu wyjściowa macierz R przyjmie postać macierzy górnotrójkątnej z zerowymi wierszami, które będzie można policzyć. Permutacje wykonywane są po kolumnach i posortowane po największej normie wektora.

In [544]:
# test na predefiniowanych macierzach. 
# f: funkcja zwracającą rząd macierzy pewna metodą
# name: nazwa metody liczącej rząd macierzy
# maxError: maksymalna wartość bezwzględna interpretowana jako 0
# rng: generator liczb losowych. Można przekazać np. MT19937(<seed>) aby generować te same zestawy losowych danych
function test(f, f_name; maxError=getPrecisionU()*50, rng=nothing)
    local _rand = rng==nothing ? rand : (args...)->rand(rng, args...) # custom rand if generator is set
    @printf(">>> Random test ID: [\e[1;35m%.16f\e[m]\n", _rand())

    function _do_test(test)
        local exact, computed = haskey(test, :rank) ? test.rank : rank(test.M), f(test.M)
        local success = exact==computed
        if success
            @printf("[%5s] [exact:\t%3d] [%s:\t%3d] <<< %s\n", 
                    exact==computed ? "\e[1;32mPASS\e[m" : "\e[1;31mFAIL\e[m", exact, f_name, computed, test.desc)
        end
    end

    # test data
    local zero_100x100 = zeros(Float64, 100, 100)
    local Id_100x100 = Matrix{Float64}(I, 100, 100)
    local Diag_100x100 = Diagonal(rand(Float64, 100))
    local A = [12 -51 4; 6 167 -68; -4 24 -41]
    local rand_10x10 = _rand(-10:10, 10, 10)
    local row_same = copy(rand_10x10); row_same[2, :] = -row_same[5, :]
    local row_col_same = copy(row_same); row_col_same[:, 2] = -row_col_same[:, 4]
    #local Kahan = Matrix Diagonal(#FIXME)
    
    # tests consist of fieds M=matrix, desc=description of test and optional rank=rank of matrix(not calculated with lib)
    local tests = [
        (M=[1 1; 1 1],   desc="2x2 [1 1; 1 1] matrix")
        (M=zero_100x100, desc="Zero 100x100 matrix")
        (M=Id_100x100,   desc="Identity 100x100 matrix")
        (M=Diag_100x100, desc="Diagonal 100x100 matrix of Float64  elements")
        (M=A,            desc="Predefined [12 -51 4; 6 167 -68; -4 24 -41]")
        (M=rand_10x10,   desc="Random 10x10 matrix of integer values on [-10..10]")
        (M=row_same,     desc=" ^ one duplicate row") 
        (M=row_col_same, desc=" ^ one duplicate row and col")
        #(M=Kahan,       desc="Kahan matrix")
    ]

    @printf("> Hardcoded tests:\n")
    for test in tests
        _do_test(test)
    end
    
    flush(stdout)
    @printf("> 100 random sparse matrices of size 100x100:\n")
    for i in 1:10
        local M = zeros(Float64, 100, 100)
        local desc = "random sparse: ["
        local fields = rand(1:100)
        for field in 1:fields # insert n elements where n on [1..100]
            local row, col, val = rand(1:100), rand(1:100), rand(-100:100)
            #desc = desc*"("*string(row)*","*string(col)*","*string(val)*") " # log matrix to description in readable format
            M[row, col] = val # random column random row insert random Float64
        end
        desc = desc*string(i)*"]"
        _do_test((M=M, desc=desc))
        flush(stdout)
    end
    @printf(">>> TESTS END\n")
end

test(rankRevealHouseholder, "Hausholder")

>>> Random test ID: [0.1821285404980577]
> Hardcoded tests:
[PASS] [exact:	  1] [Hausholder:	  1] <<< 2x2 [1 1; 1 1] matrix
[PASS] [exact:	  0] [Hausholder:	  0] <<< Zero 100x100 matrix
[PASS] [exact:	100] [Hausholder:	100] <<< Identity 100x100 matrix
[PASS] [exact:	100] [Hausholder:	100] <<< Diagonal 100x100 matrix of Float64  elements
[PASS] [exact:	  3] [Hausholder:	  3] <<< Predefined [12 -51 4; 6 167 -68; -4 24 -41]
[PASS] [exact:	 10] [Hausholder:	 10] <<< Random 10x10 matrix of integer values on [-10..10]
[PASS] [exact:	  9] [Hausholder:	  9] <<<  ^ one duplicate row
[PASS] [exact:	  9] [Hausholder:	  9] <<<  ^ one duplicate row and col
> 100 random sparse matrices of size 100x100:
[PASS] [exact:	 24] [Hausholder:	 24] <<< random sparse: [8]
>>> TESTS END


Inny sposób: rozkład SVD, mogę tylko wspomnieć, bo nie jest nigdzie zbyt opisany i 